In [1]:
import random
import numpy as np, numpy.random
import math
import collections 
import bisect 
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
import os

In [2]:
class Benchmark:
    """a class that generates benmarks"""
    def __init__(self, icpc, ml, sc):
        self.ICPC = icpc
        self.ML = ml
        self.SL = 500
        self.SC = sc
        self.columns = {}
        self.motif = []

    # Generate SC random sequences (with uniform frequencies of A,C,G,T). Each random sequence has length SL.
    def generate_random_sequence(self):
        sequence = ''
        for i in range(self.SL):
            sequence += (random.choice('ACGT'))
        return sequence

    # 3) Generate a random motif (position weight matrix) of length ML, with total information content being ICPC * ML. See note on this step at the end of this section.
    def generate_random_columns(self):
        for i in range(10000):
            random_column = list(np.random.dirichlet(np.ones(4)*random.choice([0.01,0.1,1])))
            informatio_content = sum([a*math.log(a/0.25,2) if a != 0 else 0 for a in random_column])
            self.columns[informatio_content] = random_column
        pass

    def generate_random_motif(self):
        while(True):
            motif = []
            desired_col_ic = self.ICPC*self.ML
            chosen_columns = random.sample(list(self.columns.items()),self.ML)
            for item in chosen_columns:
                desired_col_ic -= item[0]
                motif.append(item[1])
            if desired_col_ic <= 0.1 and desired_col_ic >= 0:
                self.motif = motif
                return motif

    # 4) Generate SC strings of length ML each, by sampling from this random motif. Each of these strings will be called a ‘site’. Note: to sample a string from a given motif, consider each column of the motif, from left to right, sample a character according to the probability distribution prescribed by that column, and keep appending to the right end of a string. If this is not clear, ask the TAs. 
    def generate_site(self):
        site = ''
        for i in range(self.ML):
            a = random.random()
            if a <= self.motif[i][0]:
                site += 'A'
            elif a <= self.motif[i][0] + self.motif[i][1]:
                site += 'C'
            elif a <= self.motif[i][0] + self.motif[i][1] + self.motif[i][2]:
                site += 'G'
            elif a <=  1:
                site += 'T'
        return site




In [3]:
def create_benchmark(icpc, ml, sc):
    for k in range(10):
        dir = "param" + str([icpc, ml, sc]) + "/trial" + str(k) +  "/"
        path = dir
        if not os.path.exists(path):
            os.makedirs(path)
        benchmark = Benchmark(icpc, ml, sc)
        benchmark.generate_random_columns()
        benchmark.generate_random_motif()
        sequences = []
        locations = []
        
        for i in range(benchmark.SC):
            sequence = benchmark.generate_random_sequence()
            site = benchmark.generate_site()
            # plant site
            location = random.randrange(benchmark.SL - benchmark.ML)
            sequence = sequence[:location] + site + sequence[location+benchmark.ML:]
            sequences.append(sequence)
            locations.append(location)
        
        records = (SeqRecord(Seq(seq, IUPAC.IUPACData), str(index)) for index,seq in enumerate(sequences) )
        SeqIO.write(records, dir + "sequences.fa", "fasta")

        f = open(dir +'sites.txt','w')
        for ele in locations:
            f.write(str(ele)+'\n')
        f.close()

        f = open(dir +'motif.txt','w')
        f.write("MOTIF"+ str(k) + " "+ str(benchmark.ML) + '\n')
        for ele in benchmark.motif:
            a = list([str(a) for a in ele])
            f.write(' '.join(a)+'\n')
        f.close()

        f = open(dir +'motiflength.txt','w')
        f.write(str(benchmark.ML))
        f.close()
    
    pass

In [4]:
create_benchmark(1.5,7,10)

create_benchmark(1,7,10)
create_benchmark(2,7,10)

create_benchmark(1.5,6,10)
create_benchmark(1.5,8,10)

create_benchmark(1.5,7,5)
create_benchmark(1.5,7,20)